In [ ]:
import numpy as np
import cv2
import os

In [ ]:
def downsampling_image(image, reduction_scale) :
    for i in range(0, reduction_scale) :
        if len(image.shape) > 2:
            row, col = image.shape[:2]
        else :
            row, col = image.shape
        
        image = cv2.pyrDown(image,dstsize=(col//2,row//2))
    return image

In [ ]:
def smoothing(img):
    img = cv2.GaussianBlur(img,(9,9),cv2.BORDER_DEFAULT)
    return (img)

In [ ]:
def sharpening(frame):
    kernel = np.array([ [0, 0, 0],
                        [0, 2, 0],
                        [0, 0, 0] ])
    frame = cv2.filter2D(frame, -1, kernel)
    return (frame)

In [ ]:
def preprocessing(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return (img, img_gray)

In [ ]:
params_dir = os.getcwd()+'/camera_params/'  
print(params_dir)

ret = np.load(params_dir+'ret.npy')
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')
focal_length = (K[0,0] + K[1,1])/2

print(K)

In [ ]:
win_size = 5
min_disparity = -1
max_disparity = 63
num_disparity = max_disparity - min_disparity 

In [ ]:
stereo = cv2.StereoSGBM_create(
    minDisparity = min_disparity, 
    numDisparities = num_disparity,
    blockSize = 20,
    uniquenessRatio = 1,
    speckleWindowSize = 20,
    speckleRange = 5,
    disp12MaxDiff = 2,
    P1 = 8*3*win_size**2,
    P2 = 32*3*win_size**2
    )

In [ ]:
template_list = list()
template_gray_list = list()

for i in range(1,5):
    template = cv2.imread('./images/final_exam/Templates/Template-{}.png'.format(i))
    template = smoothing(template)
    template, template_gray = preprocessing(template)
    template_list.append(template)
    template_gray_list.append(template_gray)

In [ ]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [ ]:
left_cap = cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi')
right_cap = cv2.VideoCapture('./videos/final_exam/Dataset-1/right_output-1.avi')

ret ,left_img = left_cap.read()
ret ,right_img = right_cap.read() 

h, w = left_img.shape[:2]

new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w,h),1, (w,h))

while (left_cap.isOpened() and right_cap.isOpened() ) :
    
    ret_left, left_img = left_cap.read()
    ret_right, right_img = right_cap.read()
    
    if ret_left and ret_right:
        
        left_img_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
        right_img_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

        left_img_undistorted = cv2.undistort(left_img_gray, K, dist, None, new_camera_matrix)
        right_img_undistorted = cv2.undistort(right_img_gray, K, dist, None, new_camera_matrix)
        
        left_img_down = downsampling_image(left_img_undistorted,1)
        right_img_down = downsampling_image(right_img_undistorted,1)
        
        disparity_map = stereo.compute(left_img_down, right_img_down)
        norm_disp = cv2.normalize(disparity_map, None , alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        row, col = norm_disp.shape[:2]
        norm_disp = cv2.pyrUp(norm_disp,dstsize=(col*2, row*2))
        dist_hsv = cv2.applyColorMap(norm_disp,cv2.COLORMAP_JET)

        z = focal_length/disparity_map
        z = cv2.resize(z, (left_img.shape[1],left_img.shape[0]))

        # cv2.imshow('Disparity', dist_hsv)
        # cv2.imshow('Z', z)

        sharpened = sharpening(left_img)
        query_frame, frame_gray = preprocessing(sharpened)
        frame_kpts, frame_desc = sift.detectAndCompute(frame_gray, None)

        template_kpts = list()
        matches_list = list()
        good_matches = list()
        good_matches_list = list()

        for template_gray in template_gray_list:
            img_kpts, img_desc = sift.detectAndCompute(template_gray, None)

            matches = bf.knnMatch(img_desc, frame_desc, k=2)
            
            Good_matches = list()
            Good_matches_list = list()

            for m, n in matches :
                if m.distance < 0.65*n.distance:
                    Good_matches.append(m)
                    Good_matches_list.append([m])
            
            template_kpts.append(img_kpts)
            matches_list.append(len(Good_matches))
            good_matches.append(Good_matches)
            good_matches_list.append(Good_matches_list)

            index = matches_list.index(max(matches_list))

        MIN_MATCH_NUMBER = 10

        if len(good_matches[index]) > MIN_MATCH_NUMBER:
            tp = np.float32([ template_kpts[index][m.queryIdx].pt for m in good_matches[index] ]).reshape(-1,1,2)
            fp = np.float32([ frame_kpts[m.trainIdx].pt for m in good_matches[index] ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(tp, fp, cv2.RANSAC)

            h, w = template_list[index].shape[:2]
            template_point = np.float32([[0,0], [0,h-1], [w-1,h-1], [w-1,0]]).reshape(-1,1,2)
            
            dst_point = cv2.perspectiveTransform(template_point, H)
            
            cX = ((dst_point[0,0][0]+ dst_point[2,0][0])/2).astype(int)
            cY = ((dst_point[0,0][1]+ dst_point[2,0][1])/2).astype(int)

            cv2.polylines(left_img, [np.int32(dst_point)], True, (0,0,255), 3, cv2.LINE_AA)

            X = (cX*z[cY, cX])/focal_length
            Y = (cY*z[cY, cX])/focal_length
            Z = z[cY, cX]

            text = 'X:{:.1f} Y:{:.1f} Z:{:.1f}'.format(X-0.5, Y-0.5, Z)
            cv2.putText(left_img, text, [cX, cY], cv2.FONT_HERSHEY_COMPLEX, 0.75, (0,255,0), 2)

            cv2.imshow('detected frame', left_img)
        else:
            cv2.imshow('detected frame', left_img) 
        
        key = cv2.waitKey(33) & 0xFF
        if key == 27 or key == ord('q') :
            break
    else:
        break

left_cap.release()
right_cap.release()
cv2.destroyAllWindows()